In [12]:
import os, sys, re
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import networkx as nx

from tqdm import tqdm
from pyvis.network import Network

In [13]:
provinces = ['Gilan', 'Golestan', 'Mazandaran']

In [41]:
# Unzip the data
os.system("unzip mapped-data.zip")

Archive:  mapped-data.zip
   creating: mapped-data/
  inflating: mapped-data/.DS_Store   
  inflating: __MACOSX/mapped-data/._.DS_Store  
   creating: mapped-data/Golestan/
   creating: mapped-data/Mazandaran/
   creating: mapped-data/Gilan/
   creating: mapped-data/Golestan/Hourly/
  inflating: mapped-data/Golestan/.DS_Store  
  inflating: __MACOSX/mapped-data/Golestan/._.DS_Store  
   creating: mapped-data/Golestan/Daily/
   creating: mapped-data/Mazandaran/Hourly/
  inflating: mapped-data/Mazandaran/.DS_Store  
  inflating: __MACOSX/mapped-data/Mazandaran/._.DS_Store  
   creating: mapped-data/Mazandaran/Daily/
   creating: mapped-data/Gilan/Hourly/
  inflating: mapped-data/Gilan/.DS_Store  
  inflating: __MACOSX/mapped-data/Gilan/._.DS_Store  
   creating: mapped-data/Gilan/Daily/
  inflating: mapped-data/Golestan/Hourly/شا�?ر�?د - ا�?زادش�?ر.csv  
  inflating: mapped-data/Golestan/Hourly/کردک�?�? - ب�?در ترک�?�?.csv  
  inflating: mapped-data/Golestan/Hourly/گ�?بد - ا�?�? �?�?ا.cs

0

In [42]:
# Create a directory data
if not os.path.exists("data"):
    os.makedirs("data")

# Create folders of provinces
for province in provinces:
    if not os.path.exists(f"data/{province}"):
        os.makedirs(f"data/{province}")

# Copy all files from ./mapped-data/{province}/Daily to ./data/{province}/
for province in provinces:
    os.system(f"cp -r mapped-data/{province}/Daily/* data/{province}/")

In [43]:
# Remove the folder mapped-data and __MACOSX
os.system("rm -r mapped-data __MACOSX")

for province in provinces:
    os.system(f"rm ./data/{province}/.DS_Store")

rm: ./data/Gilan/.DS_Store: No such file or directory
rm: ./data/Golestan/.DS_Store: No such file or directory
rm: ./data/Mazandaran/.DS_Store: No such file or directory


In [27]:
# Read city-translation.csv
city_translation = pd.read_csv("./helpers/city-translation.csv")
road_translation = pd.read_csv("./helpers/road-translation.csv")

In [48]:
for province in provinces: 
    files = os.listdir(f'./data/{province}')
    for file in files:
        df = pd.read_csv(f'./data/{province}/{file}')
        # If df has 16 columns, pass
        if df.shape[1] == 16 or df.shape[1] == 15:
            continue
        if df.shape[1] == 20:
            # Combine "مدت زمان کارکرد (دقیقه)" and "مدت زمان کارکرد(دقیقه)"
            df['مدت زمان کارکرد (دقیقه)'] = df['مدت زمان کارکرد (دقیقه)'].fillna(0)
            df['مدت زمان کارکرد(دقیقه)'] = df['مدت زمان کارکرد(دقیقه)'].fillna(0)
            df['مدت زمان کارکرد (دقیقه)'] = df['مدت زمان کارکرد (دقیقه)'] + df['مدت زمان کارکرد(دقیقه)']
            df = df.drop(columns=['مدت زمان کارکرد(دقیقه)'])

            # Combine "تعداد تخلف سرعت غیرمجاز" and "تعداد تخلف سرعت غیر مجاز"
            df['تعداد تخلف سرعت غیرمجاز'] = df['تعداد تخلف سرعت غیرمجاز'].fillna(0)
            df['تعداد تخلف سرعت غیر مجاز'] = df['تعداد تخلف سرعت غیر مجاز'].fillna(0)
            df['تعداد تخلف سرعت غیرمجاز'] = df['تعداد تخلف سرعت غیرمجاز'] + df['تعداد تخلف سرعت غیر مجاز']
            df = df.drop(columns=['تعداد تخلف سرعت غیر مجاز'])

            # Combine "تعداد تخلف سبقت غیرمجاز" and "تعداد تخلف سبقت غیر مجاز"
            df['تعداد تخلف سبقت غیرمجاز'] = df['تعداد تخلف سبقت غیرمجاز'].fillna(0)
            df['تعداد تخلف سبقت غیر مجاز'] = df['تعداد تخلف سبقت غیر مجاز'].fillna(0)
            df['تعداد تخلف سبقت غیرمجاز'] = df['تعداد تخلف سبقت غیرمجاز'] + df['تعداد تخلف سبقت غیر مجاز']
            df = df.drop(columns=['تعداد تخلف سبقت غیر مجاز'])

            # Combine "تعداد تخلف فاصله غیر مجاز" and "تعداد تخلف فاصله غیرمجاز"
            df['تعداد تخلف فاصله غیرمجاز'] = df['تعداد تخلف فاصله غیرمجاز'].fillna(0)
            df['تعداد تخلف فاصله غیر مجاز'] = df['تعداد تخلف فاصله غیر مجاز'].fillna(0)
            df['تعداد تخلف فاصله غیرمجاز'] = df['تعداد تخلف فاصله غیرمجاز'] + df['تعداد تخلف فاصله غیر مجاز']
            df = df.drop(columns=['تعداد تخلف فاصله غیر مجاز'])
        # Save to csv
        print(f'File {province}/{file} has been corrected.')
        df.to_csv(f'./data/{province}/{file}', index=False)

In [53]:
# Create a set to hold all column names
columns = set()

for province in provinces:
  files = os.listdir(f'./data/{province}')
  for file in files:
    df = pd.read_csv(f'./data/{province}/{file}')
    columns.update(df.columns)

# Print the length of columns
print(len(columns))
print(list(columns))

16
['Number of Class 3 Vehicles', 'Number of Illegal Distance Violations', 'Road Name', 'Number of Illegal Overtaking Violations', 'Number of Class 4 Vehicles', 'Number of Class 1 Vehicles', 'Number of Class 5 Vehicles', 'Estimated Number', 'Average Speed', 'Length of Operation (minutes)', 'Start Time', 'End Time', 'Number of Class 2 Vehicles', 'Road Code', 'Total Number of Vehicles', 'Number of Illegal Speed Violations']


In [50]:
# Create a dictionary to change columns
columns_dict = {
  'کد محور': 'Road Code',
  'نام محور': 'Road Name',
  'زمان شروع': 'Start Time',
  'زمان پایان': 'End Time',
  'مدت زمان کارکرد (دقیقه)': 'Length of Operation (minutes)',
  'مدت زمان کارکرد(دقیقه)': 'Length of Operation (minutes)',
  'تعداد برآورد شده': 'Estimated Number',
  'تعداد تخلف سرعت غیرمجاز': 'Number of Illegal Speed Violations',
  'تعداد تخلف سرعت غیر مجاز': 'Number of Illegal Speed Violations',
  'تعداد تخلف فاصله غیرمجاز': 'Number of Illegal Distance Violations',
  'تعداد تخلف فاصله غیر مجاز': 'Number of Illegal Distance Violations',
  'تعداد تخلف سبقت غیرمجاز': 'Number of Illegal Overtaking Violations',
  'تعداد تخلف سبقت غیر مجاز': 'Number of Illegal Overtaking Violations',
  'تعداد وسیله نقلیه کلاس 1':  'Number of Class 1 Vehicles',
  'تعداد وسیله نقلیه کلاس 2': 'Number of Class 2 Vehicles',
  'تعداد وسیله نقلیه کلاس 3': 'Number of Class 3 Vehicles',
  'تعداد وسیله نقلیه کلاس 4': 'Number of Class 4 Vehicles',
  'تعداد وسیله نقلیه کلاس 5': 'Number of Class 5 Vehicles',
  'تعداد کل وسیله نقلیه': 'Total Number of Vehicles',
  'سرعت متوسط': 'Average Speed',
}

In [51]:
for province in provinces:
  files = os.listdir(f'./data/{province}')
  for file in files:
    df = pd.read_csv(f'./data/{province}/{file}')
    df = df.rename(columns=columns_dict)
    df.to_csv(f'./data/{province}/{file}', index=False)

In [52]:
for province in provinces:
  files = os.listdir(f'./data/{province}')
  for file in files:
    df = pd.read_csv(f'./data/{province}/{file}')
    if df['Road Code'].nunique() != 1:
      print(f'File {province}/{file} has inconsistency in road code.')

In [75]:
# Create a dataframe to hold road codes for each file
road_codes = pd.DataFrame(columns=['Province', 'File', 'Road Code'])

for province in provinces:
  files = os.listdir(f'./data/{province}')
  for file in files:
    df = pd.read_csv(f'./data/{province}/{file}')
    road_code = df['Road Code'].unique()[0]
    road_codes = pd.concat([road_codes, pd.DataFrame([[province, file, road_code]], columns=['Province', 'File', 'Road Code'])])

In [68]:
# Check if roads with same "Road Code" belong to different provinces
for road_code in road_codes['Road Code'].unique():
  provinces = road_codes[road_codes['Road Code'] == road_code]['Province'].unique()
  if len(provinces) > 1:
    print(f'Road Code {road_code} belongs to different provinces.')

In [74]:
# For each value of Road Code that is not unique, print the rows
for road_code in road_codes['Road Code'].unique():
  if road_codes[road_codes['Road Code'] == road_code].shape[0] > 1:
    display(road_codes[road_codes['Road Code'] == road_code])
    # Create a list of file names
    files = road_codes[road_codes['Road Code'] == road_code]['File'].values
    # Print each file with their relative index name
    for i, file in enumerate(files):
      print(f'{i}: {file}')
    # Get user input on which file to keep
    file_to_keep = int(input('Enter the index of the file to keep: '))

    if (file_to_keep == -1):
      break
    
    print(files[file_to_keep])
    province = road_codes[road_codes['Road Code'] == road_code]['Province'].values[file_to_keep]
    # Open the given file
    df = pd.read_csv(f'./data/{province}/{files[file_to_keep]}')
    # Open the other files and append them to the given file
    for i, file in enumerate(files):
      if i == file_to_keep:
        continue
      df_temp = pd.read_csv(f'./data/{province}/{file}')
      df = pd.concat([df, df_temp])
      # Remove the file
      os.remove(f'./data/{province}/{file}')
    # Save the file
    df.to_csv(f'./data/{province}/{files[file_to_keep]}', index=False)


,Province,File,Road Code
0,Gilan,اردبيل - آستارا.csv,543251
0,Gilan,آق چاي - آستارا (اردبيل - آستارا).csv,543251


0: اردبيل - آستارا.csv
1: آق چاي - آستارا (اردبيل - آستارا).csv
آق چاي - آستارا (اردبيل - آستارا).csv


,Province,File,Road Code
0,Gilan,لوشان - قزوین.csv,543502
0,Gilan,لوشان - قزوین (قدیم).csv,543502


0: لوشان - قزوین.csv
1: لوشان - قزوین (قدیم).csv
لوشان - قزوین (قدیم).csv


,Province,File,Road Code
0,Gilan,آستارا - اردبيل.csv,543201
0,Gilan,آستارا - آق چاي (آستارا - اردبيل).csv,543201


...

0: پلور - گزنک (رودهن - آمل).csv
1: پلور - گزنک.csv
پلور - گزنک (رودهن - آمل).csv


,Province,File,Road Code
0,Mazandaran,گزنک - پلور (آمل - رودهن).csv,163258
0,Mazandaran,گزنک - پلور.csv,163258


0: گزنک - پلور (آمل - رودهن).csv
1: گزنک - پلور.csv
گزنک - پلور (آمل - رودهن).csv


,Province,File,Road Code
0,Mazandaran,آمل - بابل (بزرگراه).csv,163251
0,Mazandaran,آمل - بابل.csv,163251


0: آمل - بابل (بزرگراه).csv
1: آمل - بابل.csv
آمل - بابل (بزرگراه).csv


In [5]:
# Read all the files
for province in provinces:
  files = os.listdir(f'./data/{province}')
  for file in files:
    # Extract the content between parentheses
    road_name = re.search(r'\((.*?)\)', file)
    # If there is no parentheses, pass
    if road_name is None:
      continue
    road_name = road_name.group(1)
    if road_name in list(road_translation['Persian']):
      continue
    else:
      print(road_name)
      res = input('Enter the correct road name: ')
      if (res == -1):
        break
      # If res is in road_translation['Persian']
      if res in list(road_translation['Persian']):
        # Change the file name
        os.rename(f'./data/{province}/{file}', f'./data/{province}/{file.replace(road_name, res)}')
      else:
        # Get the English Name
        res = input('Enter the English name: ')
        # Append to road_translation
        road_translation = pd.concat([road_translation, pd.DataFrame([[road_name, res, province]], columns=['Persian', 'English', 'Province'])])

In [7]:
# Reset index of road_translation and save the file
road_translation = road_translation.reset_index(drop=True)
road_translation.to_csv('./helpers/road-translation.csv', index=False)

In [17]:
for province in provinces:
  files = os.listdir(f'./data/{province}')
  for file in files:
    # Remove the parentheses and the content inside them
    temp = re.sub(r'\([^)]*\)', '', file)
    # Remove .csv
    temp = temp.replace('.csv', '')

    starting_node = temp.split('-')[0].strip()
    ending_node = temp.split('-')[1].strip()

    if starting_node in list(city_translation['Persian']):
      continue
    else:
      print(starting_node)
      res = input('Enter the correct city name: ')
      if (res == -1):
        break
      if res in list(city_translation['Persian']):
        os.rename(f'./data/{province}/{file}', f'./data/{province}/{file.replace(starting_node, res)}')
      else:
        res = input('Enter the English name: ')
        city_translation = pd.concat([city_translation, pd.DataFrame([[starting_node, res, province]], columns=['Persian', 'English', 'Province'])])
    
    if ending_node in list(city_translation['Persian']):
      continue
    else:
      print(ending_node)
      res = input('Enter the correct city name: ')
      if (res == -1):
        break
      if res in list(city_translation['Persian']):
        os.rename(f'./data/{province}/{file}', f'./data/{province}/{file.replace(ending_node, res)}')
      else:
        res = input('Enter the English name: ')
        city_translation = pd.concat([city_translation, pd.DataFrame([[ending_node, res, province]], columns=['Persian', 'English', 'Province'])])

In [18]:
# Reset index of city_translation and save the file
city_translation = city_translation.reset_index(drop=True)
city_translation.to_csv('./helpers/city-translation.csv', index=False)

In [29]:
# Make data zip folder
os.system('zip -r data.zip data')

  adding: data/ (stored 0%)
  adding: data/Golestan/ (stored 0%)
  adding: data/Golestan/کردکوي - بندر ترکمن.csv (deflated 78%)
  adding: data/Golestan/گنبد - آق قلا.csv (deflated 76%)
  adding: data/Golestan/مينودشت - گنبد.csv (deflated 77%)
  adding: data/Golestan/آزادشهر - خوش ییلاق (آزادشهر - شاهرود).csv (deflated 82%)
  adding: data/Golestan/سه راهي کلاله - تنگراه.csv (deflated 79%)
  adding: data/Golestan/کردکوي - گرگان (قدیم).csv (deflated 79%)
  adding: data/Golestan/گاليکش - مينودشت.csv (deflated 77%)
  adding: data/Golestan/بندر ترکمن - سه راهي يساقي.csv (deflated 80%)
  adding: data/Golestan/آزادشهر - گنبد.csv (deflated 77%)
  adding: data/Golestan/آق قلا - گنبد.csv (deflated 75%)
  adding: data/Golestan/آزادشهر - مينودشت.csv (deflated 78%)
  adding: data/Golestan/گرگان - علي آباد.csv (deflated 77%)
  adding: data/Golestan/گنبد - مينودشت.csv (deflated 78%)
  adding: data/Golestan/کردکوي - گلوگاه.csv (deflated 76%)
  adding: data/Golestan/آق قلا - بندر ترکمن.csv (deflated 7

0